In [1]:
import torch
import os
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, random_split
from torchvision import datasets, transforms, models
from torchvision.transforms import v2
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor
import matplotlib.pyplot as plt
from torchvision.io import read_image
from torch.utils.tensorboard import SummaryWriter

In [2]:
num_to_label= {
    0: "backpack",
    1: "book",
    2: "car",
    3: "pizza",
    4: "sandwich",
    5: "snake",
    6: "sock",
    7: "tiger",
    8: "tree",
    9: "watermelon"
}

label_to_num = dict((v,k) for k,v in num_to_label.items())

# Obtains all the labels from a root directory.
def obtain_labels(root_dir):
    labels = []
    # Find all files contained in the root directory
    walk = os.walk(root_dir, topdown=False)
    for root, _, files in walk:
        # Ignore the root directory
        if root_dir == root: continue
        # Extract the label from the current directory
        root = os.path.basename(root)
        # Concatenate the labels from the curr. directory.
        labels += list(map(lambda x: [x, root], files))

    return np.array(labels)

class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_labels = obtain_labels(img_dir)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels[idx, 1], self.img_labels[idx, 0])
        image = Image.open(img_path)
        label = self.img_labels[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

class LabelToNum(torch.nn.Module):
    def forward(self, label):
        return label_to_num[label]

label_transform = LabelToNum()

dataset_transform = v2.Compose([
    v2.Resize(size=(224, 224)),
    v2.RandomHorizontalFlip(),
    v2.RandomVerticalFlip(),
    v2.PILToTensor(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

c:\Users\Henry Chan\anaconda3\envs\mlearn\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [3]:
def split_valid(dataset, validation_split=0.2, random_seed=42):
    # Creating data indices for training and validation splits.
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    # Shuffle the indices based on a random seed.
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    # Split the training and validation indices
    train_indices, val_indices = indices[split:], indices[:split]

    # Create the samplers for training and validation.
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)
    return train_sampler, valid_sampler

In [4]:
## Obtain Real datasets and split for validation sets.
train_dataset_real = CustomImageDataset(img_dir="data/real_train", transform=dataset_transform, target_transform=label_transform)
test_dataset_real = CustomImageDataset(img_dir="data/real_test", transform=dataset_transform, target_transform=label_transform)

## Obtain Sketch datasets and split for validation sets.
train_dataset_sketch = CustomImageDataset(img_dir="data/sketch_train", transform=dataset_transform, target_transform=label_transform)
test_dataset_sketch = CustomImageDataset(img_dir="data/sketch_test", transform=dataset_transform, target_transform=label_transform)

t_real_sample, v_real_sample = split_valid(train_dataset_real)
t_sketch_sample, v_sketch_sample = split_valid(train_dataset_sketch)

In [5]:
# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

# Training init
learning_rate = 1e-3
epochs = 25

real_size = len(train_dataset_real)
sketch_size = len(train_dataset_sketch)
batch_ratio = real_size/sketch_size
batch_sketch_size = 50
batch_size = {'real': int(batch_sketch_size*batch_ratio), 'sketch':batch_sketch_size}


cuda


In [6]:
## Real domain DataLoaders
# DataLoader for Training
dl_train_real = DataLoader(train_dataset_real, batch_size=batch_size['real'], sampler=t_real_sample)
# DataLoader for Validation
dl_valid_real = DataLoader(train_dataset_real, batch_size=batch_size['real'], sampler=v_real_sample)
# DataLoader for Testing
dl_test_real = DataLoader(test_dataset_real, batch_size=batch_size['real'], shuffle=True)

## Sketch domain DataLoaders
# DataLoader for Training
dl_train_sketch = DataLoader(train_dataset_sketch, batch_size=batch_size['sketch'], sampler=t_sketch_sample)
# DataLoader for Validation
dl_valid_sketch = DataLoader(train_dataset_sketch, batch_size=batch_size['sketch'], sampler=v_sketch_sample)
# DataLoader for Testing
dl_test_sketch = DataLoader(test_dataset_sketch, batch_size=batch_size['sketch'], shuffle=True)

In [7]:
def train(src_loader, model, loss_fn, optimizer, lr_optimizer, epoch, device):
    running_loss = 0
    train_loss = 0

    num_correct = 0
    num_samples = 0
   
    for idx, (data, targets) in enumerate(src_loader):
        # Map the data and targets to GPU.
        data = data.to(device)
        targets = targets.to(device)

        # Predict labels using the model
        scores = model(data)
        # Calculate the cross-entropy loss
        loss = loss_fn(scores, targets)
        
        optimizer.zero_grad()
        loss.backward()

        # update the parameters according to gradients
        optimizer.step()

        # Gather correct predictions done by the model to determine training accuaracy.
        _, predictions = scores.max(1)
        num_correct += (predictions == targets).sum()
        num_samples += predictions.size(0)

        # Gather data and report
        running_loss += loss.item()
        train_loss += loss.item()

        
        # report every 100 iterations
        if idx % 25 == 24:
            print(' epoch {} loss: {:.4f}'.format(epoch+1, running_loss / 25))
            running_loss = 0.0

    
    print(optimizer.param_groups[0]["lr"])
    lr_optimizer.step()

    train_loss = train_loss/(idx+1)
    train_acc = num_correct/num_samples

    print(f'Epoch [{epoch+1}], '
          f'Train Loss: {train_loss:.4f}, ')
    print(f'{num_correct}/{num_samples} train accuracy {num_correct/num_samples}')
    
    return train_loss, train_acc


def test(src_loader, model, loss_fn, device):

    valid_loss = 0
    num_correct = 0
    num_samples = 0

    with torch.no_grad():
        for data, targets in src_loader:
            data = data.to(device)
            targets = targets.to(device)

            scores = model(data)
            _, predictions = scores.max(1)
            num_correct += (predictions == targets).sum()
            num_samples += predictions.size(0)

            valid_loss += loss_fn(scores, targets)
        
        print(f'{num_correct}/{num_samples} accuracy {num_correct/num_samples}')

    # Gather data and report
    valid_loss /= len(src_loader)
    accuracy = num_correct/num_samples
    print("Test Error: \n   Accuracy: {:.2f}, Avg loss: {:.4f} \n".format(100*accuracy, valid_loss))
    
    return valid_loss, accuracy

### Training ResNet-34 model on the Sketch domain and evaluations on the Sketch domain.

In [8]:
rn34_model = models.resnet34(weights='DEFAULT')
num_ftrs = rn34_model.fc.in_features
rn34_model.fc = torch.nn.Linear(num_ftrs, 10)

# Freeze all gradients on parameters.
for p in rn34_model.parameters():
    p.requires_grad = False

# Thaw gradients for layer 4.
for p in rn34_model.layer4.parameters():
    p.requires_grad = True

In [9]:
rn34_model.to(device)

## Optimizer construction using Adam.
params = [p for p in rn34_model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(
    params,
    lr=learning_rate,
)

lr_optimizer = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=int(epochs*0.9))


loss_fn = torch.nn.CrossEntropyLoss()

# ******************* TensorBoard *******************
writer = SummaryWriter(comment="SKETCH-ass2")

# ******************* Optimization *******************
best_accuracy = 0
for epoch in range(epochs):
    # train loop
    # set the module in training mode.
    rn34_model.train()
    train_loss, train_acc = train(dl_train_sketch, rn34_model, loss_fn, optimizer, lr_optimizer, epoch, device)
    # save to tensorboard
    writer.add_scalar('train_loss', train_loss, epoch)
    writer.add_scalar('train_acc', train_acc, epoch)

    # save model weights
    save_path = 'ckpt_{:04d}.pth'.format(epoch+1)
    torch.save(rn34_model.state_dict(), save_path)

    # validation loop
    # set the module in evaluation mode.
    rn34_model.eval()
    valid_loss, valid_accuracy = test(dl_valid_sketch, rn34_model, loss_fn, device)
    # save to tensorboard
    writer.add_scalar('valid_loss', valid_loss, epoch)
    writer.add_scalar('valid_accuracy', valid_accuracy, epoch)

    writer.add_scalars('losses', {'train_loss':train_loss,
                                  'valid_loss':valid_loss}, epoch)
    
    writer.add_scalars('accuracy', {'train_acc':train_acc,
                                  'valid_acc':valid_accuracy}, epoch)

    if valid_accuracy > best_accuracy:    # save the model with best validation accuracy
        save_path = 'ckpt_best.pth'
        torch.save(rn34_model.state_dict(), save_path)
        best_accuracy = valid_accuracy
    
    print(f'Epoch: {epoch+1}', f'Train_loss: {train_loss}', f'Valid_loss: {valid_loss}', f'Valid_acc: {valid_accuracy}')

writer.close()
print("Finished Training.")

 epoch 1 loss: 0.9715
0.001
Epoch [1], Train Loss: 0.9579, 
1085/1565 train accuracy 0.6932907104492188
304/391 accuracy 0.7774935960769653
Test Error: 
   Accuracy: 77.75, Avg loss: 1.1574 

Epoch: 1 Train_loss: 0.9579096520319581 Valid_loss: 1.1574413776397705 Valid_acc: 0.7774935960769653
 epoch 2 loss: 0.5170
0.0009949107209404665
Epoch [2], Train Loss: 0.5427, 
1316/1565 train accuracy 0.8408945798873901
322/391 accuracy 0.8235294222831726
Test Error: 
   Accuracy: 82.35, Avg loss: 0.6617 

Epoch: 2 Train_loss: 0.5427498538047075 Valid_loss: 0.6616586446762085 Valid_acc: 0.8235294222831726
 epoch 3 loss: 0.3743
0.000979746486807249
Epoch [3], Train Loss: 0.3915, 
1371/1565 train accuracy 0.8760383129119873
326/391 accuracy 0.833759605884552
Test Error: 
   Accuracy: 83.38, Avg loss: 0.5681 

Epoch: 3 Train_loss: 0.39153885561972857 Valid_loss: 0.5680795311927795 Valid_acc: 0.833759605884552
 epoch 4 loss: 0.2946
0.0009548159976772593
Epoch [4], Train Loss: 0.2928, 
1411/1565 train

In [10]:
rn34_model.to(device)
rn34_model.eval()
valid_loss, valid_accuracy = test(dl_test_sketch, rn34_model, loss_fn, device)

728/841 accuracy 0.8656361103057861
Test Error: 
   Accuracy: 86.56, Avg loss: 0.5508 



### Training ResNet-34 model on the Real domain and evaluations on the Sketch domain.

In [11]:
rn34_model = models.resnet34(weights='DEFAULT')
num_ftrs = rn34_model.fc.in_features
rn34_model.fc = torch.nn.Linear(num_ftrs, 10)

# Freeze all gradients on parameters.
for p in rn34_model.parameters():
    p.requires_grad = False

# Thaw gradients for layer 4.
for p in rn34_model.layer4.parameters():
    p.requires_grad = True

In [12]:
rn34_model.to(device)

## Optimizer construction using Adam.
params = [p for p in rn34_model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(
    params,
    lr=learning_rate,
)

lr_optimizer = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=int(epochs*0.9))

loss_fn = torch.nn.CrossEntropyLoss()

# ******************* TensorBoard *******************
writer = SummaryWriter(comment="REAL-ass2")

# ******************* Optimization *******************
best_accuracy = 0
for epoch in range(epochs):
    # train loop
    # set the module in training mode.
    rn34_model.train()
    train_loss, train_acc = train(dl_train_real, rn34_model, loss_fn, optimizer, lr_optimizer, epoch, device)
    # save to tensorboard
    writer.add_scalar('train_loss', train_loss, epoch)
    writer.add_scalar('train_acc', train_acc, epoch)

    # save model weights
    save_path = 'ckpt_{:04d}.pth'.format(epoch+1)
    torch.save(rn34_model.state_dict(), save_path)

    # validation loop
    # set the module in evaluation mode.
    rn34_model.eval()
    valid_loss, valid_accuracy = test(dl_valid_sketch, rn34_model, loss_fn, device)
    # save to tensorboard
    writer.add_scalar('valid_loss', valid_loss, epoch)
    writer.add_scalar('valid_accuracy', valid_accuracy, epoch)

    writer.add_scalars('losses', {'train_loss':train_loss,
                                  'valid_loss':valid_loss}, epoch)
    
    writer.add_scalars('accuracy', {'train_acc':train_acc,
                                  'valid_acc':valid_accuracy}, epoch)

    if valid_accuracy > best_accuracy:    # save the model with best validation accuracy
        save_path = 'ckpt_best.pth'
        torch.save(rn34_model.state_dict(), save_path)
        best_accuracy = valid_accuracy
    
    print(f'Epoch: {epoch+1}', f'Train_loss: {train_loss}', f'Valid_loss: {valid_loss}', f'Valid_acc: {valid_accuracy}')

writer.close()
print("Finished Training.")

 epoch 1 loss: 0.3546
0.001
Epoch [1], Train Loss: 0.3217, 
2864/3188 train accuracy 0.8983688950538635
185/391 accuracy 0.4731457829475403
Test Error: 
   Accuracy: 47.31, Avg loss: 2.3988 

Epoch: 1 Train_loss: 0.3216794291511178 Valid_loss: 2.3987746238708496 Valid_acc: 0.4731457829475403
 epoch 2 loss: 0.1167
0.0009949107209404665
Epoch [2], Train Loss: 0.1182, 
3076/3188 train accuracy 0.9648682475090027
178/391 accuracy 0.45524296164512634
Test Error: 
   Accuracy: 45.52, Avg loss: 2.5525 

Epoch: 2 Train_loss: 0.11822104861494154 Valid_loss: 2.5525479316711426 Valid_acc: 0.45524296164512634
 epoch 3 loss: 0.0781
0.000979746486807249
Epoch [3], Train Loss: 0.0753, 
3119/3188 train accuracy 0.9783563017845154
176/391 accuracy 0.45012786984443665
Test Error: 
   Accuracy: 45.01, Avg loss: 2.8705 

Epoch: 3 Train_loss: 0.07531936213490553 Valid_loss: 2.870485782623291 Valid_acc: 0.45012786984443665
 epoch 4 loss: 0.0402
0.0009548159976772593
Epoch [4], Train Loss: 0.0491, 
3140/3188

In [13]:
rn34_model.to(device)
rn34_model.eval()
valid_loss, valid_accuracy = test(dl_test_sketch, rn34_model, loss_fn, device)

364/841 accuracy 0.43281805515289307
Test Error: 
   Accuracy: 43.28, Avg loss: 4.5012 



In [14]:
coral_loss_weight = 10
mmd_loss_weight = 1e-2
epochs = 20
learning_rate = 1e-2

In [15]:
class MMD(torch.nn.Module):
    def forward(self, source, target):
        print(source.shape, target.shape)
        mSource = source.mean()
        mTarget = target.mean()
        L2_dist = torch.square(torch.linalg.vector_norm(mSource-mTarget))

        return L2_dist

In [16]:
class RBF(torch.nn.Module):

    def __init__(self, n_kernels=5, mul_factor=2.0, bandwidth=None):
        super().__init__()
        self.bandwidth_multipliers = mul_factor ** (torch.arange(n_kernels) - n_kernels // 2).to(device)
        self.bandwidth = bandwidth

    def get_bandwidth(self, L2_distances):
        if self.bandwidth is None:
            n_samples = L2_distances.shape[0]
            return L2_distances.data.sum() / (n_samples ** 2 - n_samples)

        return self.bandwidth

    def forward(self, X):
        L2_distances = torch.cdist(X, X) ** 2
        return torch.exp(-L2_distances[None, ...] / (self.get_bandwidth(L2_distances) * self.bandwidth_multipliers)[:, None, None]).sum(dim=0)


class MMDLoss(torch.nn.Module):

    def __init__(self, kernel=RBF()):
        super().__init__()
        self.kernel = kernel

    def forward(self, X, Y):
        K = self.kernel(torch.vstack([X, Y]))

        X_size = X.shape[0]
        XX = K[:X_size, :X_size].mean()
        XY = K[:X_size, X_size:].mean()
        YY = K[X_size:, X_size:].mean()
        return XX - 2 * XY + YY

In [17]:
class CORAL(torch.nn.Module):
    def forward(self, source, target):
        d = source.size(1)
        source_cov = torch.cov(source.t())
        target_cov = torch.cov(target.t())
        loss = torch.trace(torch.mm(source_cov-target_cov, (source_cov-target_cov).t()))
        loss = loss / (4*d*d)
        return loss

In [18]:
def train_coral(src_loader, tgt_loader, model, loss_fn, optimizer, lr_optimizer, epoch, device):
    running_loss = 0
    train_loss = 0
    
    train_steps = min(len(src_loader), len(tgt_loader))

    #mmd_fn = MMDLoss()
    coral_fn = CORAL()

    for idx in range(train_steps):
        src_input, src_label = next(iter(src_loader))
        tgt_input, _ = next(iter(tgt_loader))

        src_input = src_input.to(device)
        src_label = src_label.to(device)
        tgt_input = tgt_input.to(device)

        src_outputs = model(src_input)
        tgt_outputs = model(tgt_input)

        loss = loss_fn(src_outputs['fc'], src_label)

        #loss_mmd = mmd_fn(src_outputs['flatten'], tgt_outputs['flatten'])

        loss_coral = coral_fn(src_outputs['flatten'], tgt_outputs['flatten'])

        joint_loss = loss + coral_loss_weight*loss_coral
        print(joint_loss, loss_coral, loss)

        optimizer.zero_grad()
        joint_loss.backward()
        # update the parameters according to gradients
        optimizer.step()

        # Gather data and report
        running_loss += joint_loss.item()
        train_loss += joint_loss.item()

        # report every 100 iterations
        if idx % 25 == 24:
            print(' epoch {} loss: {:.4f}'.format(epoch+1, running_loss / 100))
            running_loss = 0.0
    
    print(optimizer.param_groups[0]["lr"])
    lr_optimizer.step()
    
    train_loss = train_loss/train_steps

    print(f'Epoch [{epoch+1}], '
          f'Train Loss: {train_loss:.4f}, ')
    
    return train_loss

def test_coral(src_loader, model, loss_fn, device):

    valid_loss = 0
    num_correct = 0
    num_samples = 0

    with torch.no_grad():
        for data, targets in src_loader:
            data = data.to(device)
            targets = targets.to(device)

            scores = model(data)['fc']
            _, predictions = scores.max(1)
            num_correct += (predictions == targets).sum()
            num_samples += predictions.size(0)

            valid_loss += loss_fn(scores, targets)
        
        print(f'{num_correct}/{num_samples} accuracy {num_correct/num_samples}')

    # Gather data and report
    valid_loss /= len(src_loader)
    accuracy = num_correct/num_samples
    print("Test Error: \n   Accuracy: {:.2f}, Avg loss: {:.4f} \n".format(100*accuracy, valid_loss))
    
    return valid_loss, accuracy

In [19]:
# MaskRCNN requires a backbone with an attached FPN
class Resnet34(torch.nn.Module):
    def __init__(self, return_nodes, num_classes):
        super(Resnet34, self).__init__()
        # Get a resnet34 backbone
        m = models.resnet34(weights='DEFAULT')
        # Print all evaluation layer names.
        _, eval_nodes = get_graph_node_names(m)
        #print(eval_nodes)
        m.fc = torch.nn.Linear(m.fc.in_features, num_classes)
        # Extract 4 main layers
        self.body = create_feature_extractor(
            m, return_nodes=return_nodes)
        
    def forward(self, x):
        x = self.body(x)
        return x

In [20]:
return_nodes = {
    'flatten':'flatten',
    'fc':'fc'
}
rn34_model = Resnet34(return_nodes, 10)

# Freeze all gradients on parameters.
for p in rn34_model.parameters():
    p.requires_grad = False

# Thaw gradients for layer 4.
for parameter in rn34_model.body.layer4.parameters():
    parameter.requires_grad = True

In [21]:
# Run on CUDA/GPU
rn34_model.to(device)

## Optimizer construction using SGD
params = [p for p in rn34_model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=learning_rate,
    weight_decay=5e-4,
    momentum=0.9
)
lr_optimizer = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=int(epochs*0.9))
loss_fn = torch.nn.CrossEntropyLoss()

# ******************* TensorBoard *******************
writer = SummaryWriter(comment="CORAL-ass2")

# ******************* Optimization *******************
best_accuracy = 0
for epoch in range(epochs):
    # train loop
    # set the module in training mode.
    rn34_model.train()
    train_loss = train_coral(dl_train_real, dl_train_sketch, rn34_model, loss_fn, optimizer, lr_optimizer, epoch, device)
    # save to tensorboard
    writer.add_scalar('train_loss', train_loss, epoch)

    # save model weights
    save_path = 'ckpt_{:04d}_22.pth'.format(epoch+1)
    torch.save(rn34_model.state_dict(), save_path)

    # validation loop
    # set the module in evaluation mode.
    rn34_model.eval()
    valid_loss, valid_accuracy = test_coral(dl_valid_sketch, rn34_model, loss_fn, device)
    # save to tensorboard
    writer.add_scalar('valid_loss', valid_loss, epoch)
    writer.add_scalar('valid_accuracy', valid_accuracy, epoch)

    writer.add_scalars('losses', {'train_loss':train_loss,
                                  'valid_loss':valid_loss}, epoch)
    
    writer.add_scalars('accuracy', {'valid_acc':valid_accuracy}, epoch)

    if valid_accuracy > best_accuracy:    # save the model with best validation accuracy
        save_path = 'ckpt_best_22.pth'
        torch.save(rn34_model.state_dict(), save_path)
        best_accuracy = valid_accuracy
    
    print(f'Epoch: {epoch+1}', f'Train_loss: {train_loss}', f'Valid_loss: {valid_loss}', f'Valid_acc: {valid_accuracy}')

writer.close()
print("Finished Training.")

tensor(2.7744, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.0050, device='cuda:0', grad_fn=<DivBackward0>) tensor(2.7244, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.6603, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.0049, device='cuda:0', grad_fn=<DivBackward0>) tensor(2.6116, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.5814, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.0045, device='cuda:0', grad_fn=<DivBackward0>) tensor(2.5362, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2430, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.0048, device='cuda:0', grad_fn=<DivBackward0>) tensor(2.1948, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.1758, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.0049, device='cuda:0', grad_fn=<DivBackward0>) tensor(2.1273, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.9465, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.0047, device='cuda:0', grad_fn=<DivBackward0>) tensor(1.8993, device='cuda:

In [23]:
rn34_model.to(device)
rn34_model.eval()
valid_loss, valid_accuracy = test_coral(dl_test_sketch, rn34_model, loss_fn, device)

494/841 accuracy 0.587395966053009
Test Error: 
   Accuracy: 58.74, Avg loss: 1.7610 

